In [94]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import pylab as plt

In [15]:
# Se cargan los datos
y_total = np.genfromtxt("msd_genre_dataset.txt",dtype =None,usecols=0,comments=None,delimiter=',',skip_header=10)
x_total = np.genfromtxt("msd_genre_dataset.txt",comments=None,delimiter=',',skip_header=10)[:,4:]

generos = [b'metal',b'punk',b'dance and electronica']

y_raw = []
x_raw = []
for i in range(0,y_total.size):
    if y_total[i] in generos:
        y_raw.append(y_total[i])
        x_raw.append(x_total[i])
        
y_raw = np.asarray(y_raw)
x_raw = np.asarray(x_raw)

In [89]:
# Preprocesamiento
x_normalizado = (x_raw - np.mean(x_raw,axis=0))/np.std(x_raw,axis=0)

# PCA
proporcion_varianza = 0
n_componentes = 0
while proporcion_varianza < 0.90:
    n_componentes = n_componentes + 1
    pca = PCA(n_components=n_componentes)
    pca = pca.fit(x_normalizado)
    proporcion_varianza = np.sum(pca.explained_variance_ratio_)
    
suma_acumulada_varianza = np.cumsum(pca.explained_variance_ratio_*100)

x_preprocesamiento = pca.fit_transform(x_normalizado)
xy_preprocesamiento = np.append(x_preprocesamiento,np.array([y_raw]).T,axis=1)

np.random.shuffle(xy_preprocesamiento)
n_prueba = round(0.3*y_raw.size)
datos_prueba = xy_preprocesamiento[0:n_prueba,:]
x_prueba = datos_prueba[:,0:-1].astype(np.float64)
y_prueba = datos_prueba[:,-1]
datos_entrenamiento = xy_preprocesamiento[n_prueba+1:,:]
x_entrenamiento = datos_entrenamiento[:,0:-1].astype(np.float64)
y_entrenamiento = datos_entrenamiento[:,-1]

In [100]:
etiquetas_prueba = np.where(y_prueba == generos[2], np.ones(y_prueba.shape), np.zeros(y_prueba.shape))
etiquetas_entrenamiento = np.where(y_entrenamiento == generos[2], np.ones(y_entrenamiento.shape), np.zeros(y_entrenamiento.shape))

In [77]:
clasificador_logreg = LogisticRegression(solver='sag')
logistic = clasificador_logreg.fit(x_entrenamiento,etiquetas_entrenamiento)

In [78]:
parametros_w_logreg = clasificador_logreg.coef_

In [82]:
etiquetas_regresion = 1/(1+np.exp(np.inner(-parametros_w_logreg,x_entrenamiento))).T

In [88]:
etiquetas_regresion[6590]

array([ 0.51117399])

In [90]:
yy = clasificador_logreg.predict(x_prueba)

In [102]:
puntaje = clasificador_logreg.score(x_prueba,etiquetas_prueba)

In [103]:
puntaje

0.48029957668511886